<a href="https://colab.research.google.com/github/EyalKoubi/CloudProject/blob/main/Hw2Dolev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install firebase

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from IPython.display import display, HTML
import IPython

import ipywidgets as widgets
from IPython.display import display
from firebase import firebase

FBconn = firebase.FirebaseApplication('https://new-proj-917e8-default-rtdb.firebaseio.com/', None)



# Function to display a multiple-choice question
def display_question(question, options):
    # HTML code for the question and options
    html_code = f'''
    <!DOCTYPE html>
    <html lang="en">
      <head>
        <meta charset="UTF-8" />
      </head>
      <body>
        <h2>{question}</h2>
        <ul>
    '''

    # Add each option to the HTML code
    for option in options:
        html_code += f'<li>{option}</li>'

    # Close the HTML code
    html_code += '''
        </ul>
      </body>
    </html>
    '''

    # Display the HTML code
    display(HTML(html_code))

# Example usage
question = "What is the capital of France?"
options = ["London", "Paris", "Berlin", "Rome"]

# Call the function to display the question
display_question(question, options)
from firebase import firebase

# Initialize the Firebase connection
firebase = firebase.FirebaseApplication('https://new-proj-917e8-default-rtdb.firebaseio.com/', None)

# Function to retrieve questions from Firebase

def retrieve_questions():
    # Retrieve the questions from the Firebase database
    result = firebase.get('/Questions', None)

    # Create an empty array to store the questions
    questions = []

    # Process the retrieved questions
    if result:
        for key, value in result.items():
            question = {
                'question': value['question'],
                'options': value['options'],
                'correctOption': value['correctOption']
            }
            questions.append(question)

    return questions

# Example usage
all_questions = retrieve_questions()


In [81]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [96]:
import random
import ipywidgets as widgets
import IPython.display as display
import os

# Initialize the Firebase connection
FBconn = firebase.FirebaseApplication('https://new-proj-917e8-default-rtdb.firebaseio.com/', None)

# Function to retrieve all questions from Firebase
def get_all_questions():
    result = FBconn.get('/Questions', None)
    questions = []
    if result:
        for key, value in result.items():
            question = {
                'question': value['question'],
                'options': value['options'],
                'correctOption': value['correctOption']
            }
            questions.append(question)
    return questions

def quizScreen():
    # Create a submit button
    submit_button = widgets.Button(description='Submit')

    # Add a question and radio buttons
    def add_question(question, options):
        question_widget = widgets.HTML(value=f"<p>{question}</p>")
        options_widget = widgets.RadioButtons(options=options, layout={'display': 'flex', 'flex_flow': 'column'})
        options_widget.index = None  # Remove default selection
        return widgets.VBox([question_widget, options_widget])

    # Create a list to store question containers
    question_containers = []

    # Get all questions
    questions = get_all_questions()

    # Shuffle the questions randomly
    random.shuffle(questions)

    # Add questions to the containers
    for question in questions:
        question_widget = add_question(question['question'], question['options'])
        question_containers.append(question_widget)

    # Set the initial question index to 0
    current_question_index = 0

    # Create a burndown graph widget
    burndown_graph = widgets.Box(layout=widgets.Layout(display='flex', flex_flow='row'))

    # Function to update the burndown graph
    def update_burndown_graph():
        burndown_icons = [widgets.Label(value='V')] * current_question_index
        burndown_graph.children = burndown_icons

    # Update the burndown graph initially
    update_burndown_graph()

    # Function to handle button click event
    def submit_button_clicked(button):
        nonlocal current_question_index

        # Get selected option
        selected_option = question_containers[current_question_index].children[1].value

        if selected_option is None:
            return

        # Check if the selected option is correct
        correct_option = questions[current_question_index]['correctOption']
        is_correct = selected_option == correct_option

        # Move to the next question
        current_question_index += 1

        if current_question_index < len(question_containers):
            display.clear_output()
            display.display(burndown_graph)  # Display burndown graph at the top
            display.display(question_containers[current_question_index])
            display.display(submit_button)
        else:
            display.clear_output()
            print("Quiz completed!")

        # Update the burndown graph
        update_burndown_graph()

    # Display the burndown graph at the top
    display.display(burndown_graph)

    # Display the first question container
    display.display(question_containers[current_question_index])

    # Display the submit button
    display.display(submit_button)

    # Register the button click event handler
    submit_button.on_click(submit_button_clicked)

# Call the quizScreen function to start the quiz
quizScreen()


Quiz completed!
